In [1]:
# Reload modulos automaticamente
%load_ext autoreload
%autoreload 2

from modules.gnn_models import GCN, GraphSAGE, GAT
from modules.graph import Graph, create_files
from modules.gnn import GNN



# from modules.gnn import GNN
# from modules.models import  ModelGraphSAGE, GraphSAGE, GCN, ModelGCN, ModelSAGESample #ModelGraphSAGE,GraphSAGE, GCN,ModelSAGESample,SAGE, ModelGCN
# from modules.predictors import DotPredictor, MLPPredictor, MLPPredictorEmbeddings
# from modules.graph_from_api import Graph_API
# from dgl.sampling import pack_traces

# import torch.nn.functional as F
# from torch.optim import Adam
# from torch.optim import SparseAdam
# import itertools
import dgl
import torch
# import matplotlib.pyplot as plt
# from utils import *
# from modules.graph import Graph
# import numpy as np
import os
# import fnmatch
# import tqdm
# import networkx as nx

# import dgl.function as fn
# from dgl.nn import DeepWalk
# from torch.utils.data import DataLoader

# from sklearn.linear_model import LogisticRegression
# from gensim.models import Word2Vec
# from sklearn.model_selection import train_test_split
# from bgp2vec.bgp2vec import BGP2VEC

/home/valentina/Desktop/GIT/TrabajoTesis/env310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


print("Versión de DGL:", dgl.__version__)
print("Versión de PyTorch:", torch.__version__)



     
TOR_LABELS_DICT = {'P2P':0, 'C2P': 1,'P2C': 2}


models = {
    'GCN': GCN,
    'GraphSAGE': GraphSAGE,
    'GAT': GAT,
}

Versión de DGL: 2.4.0+cu121
Versión de PyTorch: 2.3.0+cu121


In [3]:
# Definimos las rutas de los archivos
BASE_PATH = os.getcwd() + "/data/"
RELATIONSHIPS_FILE  = BASE_PATH + "CAIDA_AS_Relationships/Serial_1/20220701.as-rel.txt.bz2"
FEATURES_FILE = BASE_PATH + "/node_feature_mio.csv" #"/node_features.csv" 
rib_path = BASE_PATH + 'sanitized_rib.txt'
dataset_graph_path = BASE_PATH + 'dgl_graph/'
MAX_NUM_ROUTES = 1000000


## Creacion Grafo 
Creamos un grafo nx y dgl, ademas de los archivos edges.csv y nodes.csv a partir de archivos ribs previamente creados o de archivo CAIDA AS Relationships.

Crear esos archivos una unica vez con create_graph() una ves ya creados los archivos edges.csv y nodes.csv puedo ocupar directamente la funcion 

Se le puede indicar el maximo de bgp paths que se quiere (hehco para cuando se leen ribs no de caida) 

### CASO 1: RIBs
* Creacion de grafo a partir de paths recolectados de las RIBs por BGPStream
* Por ahora le asignamos a todos los nodos embeddings iniciales de de dimension 32 parte con puros 1s todos


In [5]:
# Definimos las listas de features

# features_file = 'node_degrees' #'' # 'node_degrees' # las features que se le agregaran seran 
# features_file = base_path + "/node_features.csv"
features_file = BASE_PATH + "/node_features_mio.csv"
type = 'DiGraph'#"DiGraph" # MultiDiGraph
graph_case1 = create_files(type, 
            dataset_graph_path,
            file = rib_path, 
            features_file = features_file, 
            from_caida=False, 
            remove_degree=3,
            debug=True,
            label_edges_file = RELATIONSHIPS_FILE,
            max_paths = MAX_NUM_ROUTES)
 
print('[NX Graph]',graph_case1.nx_graph)

[CARPETA CREADA]:  /home/valentina/Desktop/GIT/TrabajoTesis/data/dgl_graph/
[Creando topologia desde /home/valentina/Desktop/GIT/TrabajoTesis/data/sanitized_rib.txt]
[ARCHIVO EDGES.CSV CREADO]
[NX Graph]:  DiGraph with 43183 nodes and 150200 edges
[Agregando attr a nodos desde /home/valentina/Desktop/GIT/TrabajoTesis/data//node_features_mio.csv]
[ARCHIVO NODES.CSV CREADO], 0 nodos sin features
[NX Graph]:  DiGraph with 43183 nodes and 150200 edges
nodes.csv edges.csv
[Etiquetando aristas con CAIDA]
[INFO] Se eliminaron 12915 aristas sin 'Relationship'
[META CREADO]
[NX Graph]:  DiGraph with 26167 nodes and 120328 edges
[NX Graph]:  DiGraph with 25855 nodes and 120017 edges
[NX Graph]:  DiGraph with 25847 nodes and 120009 edges
[NX Graph] DiGraph with 25847 nodes and 120009 edges


In [ ]:
dd = 'home/valentina/Desktop/GIT/TrabajoTesis/data/CAIDA_AS_Relationships/Serial_1/20220701.as-rel.txt.bz2' \
'as-'

In [9]:
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F


In [13]:
def compute_auc(pos_score,neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)


def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    )
    return F.binary_cross_entropy_with_logits(scores, labels)


### CASO 1: 
* Encoder : GNN -> (GCN , GraphSAGE, GAT)
* Decoder : DotProduct

In [14]:
gnn = GNN(debug=True)
gnn.load_dataset(dataset_graph_path, force_reload=True)

print('[ATTR SHAPE]: ',gnn.dgl_graph.ndata['feat'].shape)
in_feats = gnn.dgl_graph.ndata['feat'].shape[1]
gnn.split_graph_edges(train_size=0.8)

Done saving data into cached files.
Graph(num_nodes=43183, num_edges=3351277,
      ndata_schemes={'feat': Scheme(shape=(69,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.float32)})
[ATTR SHAPE]:  torch.Size([43183, 69])
Generando 3351277 aristas negativas...
Aristas negativas generadas: 3351276


In [ ]:
decorer = 'DotProduct'
in_feats = gnn.dgl_graph.ndata['feat'].shape[1]
hidden_feats = 32 
out_feats = 16

for model_name in models:

    print("Training model: {}".format(model_name))

    model = models[model_name](
        in_feats=in_feats,
        hidden_feats=hidden_feats,
        out_feats=out_feats)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # ----------- training -------------------------------- #

    all_logits = []
    for e in range(50):

        # forward
        model.train()
        h = model.encode(gnn.train_g, gnn.train_g.ndata["feat"])

        pos_score = model.decodeDotProduct(gnn.train_pos_g, h)
        neg_score = model.decodeDotProduct(gnn.train_neg_g, h)


        loss = compute_loss(pos_score, neg_score)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 10 == 0:
            print("In epoch {}, loss: {}".format(e, loss))

    # ----------- 5. check results ------------------------ #
    from sklearn.metrics import roc_auc_score

    with torch.no_grad():
        pos_score = model.decodeDotProduct(gnn.test_pos_g, h)
        neg_score = model.decodeDotProduct(gnn.test_neg_g, h)
        print("AUC", compute_auc(pos_score, neg_score))
    
    # Guaradar el modelo
    torch.save(model.state_dict(), f'data/model_emb_{model_name}.pth')

    # Guardar los embeddings
    torch.save(h, f"data/embeddings_ribs_DP_{model_name}.pt")

### CASO 2: 
* Encoder : GNN -> (GCN , GraphSAGE, GAT)
* Decoder : MLP